In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [2]:
import copy
from tqdm import tqdm_notebook

import sys
sys.path.append("../..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel

In [12]:
import os
import numpy as np
import pandas as pd
import networkx as nx
from sklearn.preprocessing import LabelEncoder

In [5]:
file_folder =  '../../data/input'
os.listdir(file_folder)

['test.csv',
 'structures',
 'sample_submission.csv',
 'magnetic_shielding_tensors.csv',
 'mulliken_charges.csv',
 'potential_energy.csv',
 'scalar_coupling_contributions.csv',
 'dipole_moments.csv',
 'structures.csv',
 'train.csv']

In [8]:
train = pd.read_csv(f'{file_folder}/train.csv')
test = pd.read_csv(f'{file_folder}/test.csv')
magnetic_shielding_tensors = pd.read_csv(f'{file_folder}/magnetic_shielding_tensors.csv')
dipole_moments = pd.read_csv(f'{file_folder}/dipole_moments.csv')
mulliken_charges = pd.read_csv(f'{file_folder}/mulliken_charges.csv')
potential_energy = pd.read_csv(f'{file_folder}/potential_energy.csv')
scalar_coupling_contributions = pd.read_csv(f'{file_folder}/scalar_coupling_contributions.csv')
structures = pd.read_csv(f'{file_folder}/structures.csv')

In [10]:
atomic_radius = {'H': 0.38, 'C': 0.77, 'N': 0.75, 'O': 0.73, 'F': 0.71}
atomic_number = {'H': 1, 'C': 6, 'N': 7, 'O': 8, 'F': 9}
atomic_mass = {'H': 1.0079, 'C': 12.0107, 'N': 14.0067, 'O': 15.9994, 'F': 18.9984}
vanderwaalsradius = {'H': 120, 'C': 185, 'N': 154, 'O': 140, 'F': 135}
covalenzradius = {'H': 30, 'C': 77, 'N': 70, 'O': 66, 'F': 58}
electronegativity = {'H': 2.2, 'C': 2.55, 'N': 3.04, 'O': 3.44, 'F': 3.98}
ionization_energy = {'H': 13.5984, 'C': 11.2603, 'N': 14.5341, 'O': 13.6181, 'F': 17.4228}

In [13]:
train = pd.read_csv(f'{file_folder}/train.csv')
test = pd.read_csv(f'{file_folder}/test.csv')
sub = pd.read_csv(f'{file_folder}/sample_submission.csv')
train['type0'] = train['type'].apply(lambda x: int(x[0]))
train['type1'] = train['type'].apply(lambda x: x[1:])

le = LabelEncoder()
le.fit(train.type1.tolist())
int_bond_type = le.transform(train.type1.tolist()) 
train['int_type1']= int_bond_type

test['type0'] = test['type'].apply(lambda x: int(x[0]))
test['type1'] = test['type'].apply(lambda x: x[1:])
int_bond_type = le.transform(test.type1.tolist()) 
test['int_type1']= int_bond_type

In [21]:
train.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,type0,type1,int_type1
0,0,dsgdb9nsd_000001,1,0,1JHC,84.8076,1,JHC,0
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.2570,2,JHH,1
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.2548,2,JHH,1
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.2543,2,JHH,1
4,4,dsgdb9nsd_000001,2,0,1JHC,84.8074,1,JHC,0


In [14]:
le.classes_

array(['JHC', 'JHH', 'JHN'], dtype='<U3')

In [16]:
atomic_radius = {'H': 0.38, 'C': 0.77, 'N': 0.75, 'O': 0.73, 'F': 0.71}
atomic_number = {'H': 1, 'C': 6, 'N': 7, 'O': 8, 'F': 9}
atomic_mass = {'H': 1.0079, 'C': 12.0107, 'N': 14.0067, 'O': 15.9994, 'F': 18.9984}
vanderwaalsradius = {'H': 120, 'C': 185, 'N': 154, 'O': 140, 'F': 135}
covalenzradius = {'H': 30, 'C': 77, 'N': 70, 'O': 66, 'F': 58}
electronegativity = {'H': 2.2, 'C': 2.55, 'N': 3.04, 'O': 3.44, 'F': 3.98}
ionization_energy = {'H': 13.5984, 'C': 11.2603, 'N': 14.5341, 'O': 13.6181, 'F': 17.4228}

structures['atomic_radius'] = structures['atom'].apply(lambda x: atomic_radius[x])
structures['atomic_number'] = structures['atom'].apply(lambda x: atomic_number[x])
structures['atomic_mass'] = structures['atom'].apply(lambda x: atomic_mass[x])
structures['vanderwaalsradius'] = structures['atom'].apply(lambda x: vanderwaalsradius[x])
structures['covalenzradius'] = structures['atom'].apply(lambda x: covalenzradius[x])
structures['electronegativity'] = structures['atom'].apply(lambda x: electronegativity[x])
structures['ionization_energy'] = structures['atom'].apply(lambda x: ionization_energy[x])


In [18]:
le = LabelEncoder()
le.fit(structures.atom.tolist())
int_atom = le.transform(structures.atom.tolist()) 
structures['int_atom'] = int_atom

In [19]:
le.classes_

array(['C', 'F', 'H', 'N', 'O'], dtype='<U1')

In [20]:
structures.head()

,molecule_name,atom_index,atom,x,y,z,atomic_radius,atomic_number,atomic_mass,vanderwaalsradius,covalenzradius,electronegativity,ionization_energy,int_atom
0,dsgdb9nsd_000001,0,C,-0.012698,1.085804,0.008001,0.77,6,12.0107,185,77,2.55,11.2603,0
1,dsgdb9nsd_000001,1,H,0.002150,-0.006031,0.001976,0.38,1,1.0079,120,30,2.20,13.5984,2
2,dsgdb9nsd_000001,2,H,1.011731,1.463751,0.000277,0.38,1,1.0079,120,30,2.20,13.5984,2
3,dsgdb9nsd_000001,3,H,-0.540815,1.447527,-0.876644,0.38,1,1.0079,120,30,2.20,13.5984,2
4,dsgdb9nsd_000001,4,H,-0.523814,1.437933,0.906397,0.38,1,1.0079,120,30,2.20,13.5984,2


In [22]:
molecule_names = train.molecule_name.unique().tolist()

In [23]:
molecule_names[123]

'dsgdb9nsd_000183'

In [25]:
train[train['molecule_name']==molecule_names[123]]

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,type0,type1,int_type1
3397,3397,dsgdb9nsd_000183,6,1,2JHN,0.788742,2,JHN,2
3398,3398,dsgdb9nsd_000183,6,2,3JHC,9.182840,3,JHC,0
3399,3399,dsgdb9nsd_000183,7,1,3JHN,1.187090,3,JHN,2
3400,3400,dsgdb9nsd_000183,7,2,2JHC,-3.752440,2,JHC,0
3401,3401,dsgdb9nsd_000183,7,3,1JHC,94.230200,1,JHC,0
3402,3402,dsgdb9nsd_000183,7,5,3JHC,3.225250,3,JHC,0
3403,3403,dsgdb9nsd_000183,7,8,2JHH,-11.970200,2,JHH,1
3404,3404,dsgdb9nsd_000183,8,1,3JHN,1.187080,3,JHN,2
3405,3405,dsgdb9nsd_000183,8,2,2JHC,-3.752150,2,JHC,0
3406,3406,dsgdb9nsd_000183,8,3,1JHC,94.230000,1,JHC,0


In [26]:
structures[structures['molecule_name']==molecule_names[123]]

,molecule_name,atom_index,atom,x,y,z,atomic_radius,atomic_number,atomic_mass,vanderwaalsradius,covalenzradius,electronegativity,ionization_energy,int_atom
1833,dsgdb9nsd_000183,0,O,-0.129395,1.348488,0.088042,0.73,8,15.9994,140,66,3.44,13.6181,4
1834,dsgdb9nsd_000183,1,N,0.039735,-0.046199,0.064270,0.75,7,14.0067,154,70,3.04,14.5341,3
1835,dsgdb9nsd_000183,2,C,-1.073017,-0.632752,-0.030951,0.77,6,12.0107,185,77,2.55,11.2603,0
1836,dsgdb9nsd_000183,3,C,-2.537754,-0.284880,-0.127359,0.77,6,12.0107,185,77,2.55,11.2603,0
1837,dsgdb9nsd_000183,4,O,-2.856036,-1.696072,-0.186540,0.73,8,15.9994,140,66,3.44,13.6181,4
1838,dsgdb9nsd_000183,5,C,-1.463492,-2.087783,-0.096514,0.77,6,12.0107,185,77,2.55,11.2603,0
1839,dsgdb9nsd_000183,6,H,0.777827,1.664901,0.161506,0.38,1,1.0079,120,30,2.20,13.5984,2
1840,dsgdb9nsd_000183,7,H,-2.968264,0.212414,0.751186,0.38,1,1.0079,120,30,2.20,13.5984,2
1841,dsgdb9nsd_000183,8,H,-2.840013,0.258254,-1.031864,0.38,1,1.0079,120,30,2.20,13.5984,2
1842,dsgdb9nsd_000183,9,H,-1.125108,-2.637844,-0.983718,0.38,1,1.0079,120,30,2.20,13.5984,2


In [29]:
g = nx.Graph()
molecule_name = molecule_names[123]

In [31]:
for idx, row in structures[structures['molecule_name']==molecule_name].iterrows():
    g.add_node(row['atom_index'], x=row['x'], y=row['y'], z=row['z'], int_atom=row['int_atom'], atomic_radius=row['atomic_radius'], atomic_number=row['atomic_number'], atomic_mass=row['atomic_mass'], vanderwaalsradius=row['vanderwaalsradius'], covalenzradius=row['covalenzradius'], electronegativity=row['electronegativity'], ionization_energy=row['ionization_energy'])

In [ ]:
for idx, row in train[train['molecule_name']==molecule_name].iterrows():
    g.add_node(row['atom_index'], x=row['x'], y=row['y'], z=row['z'], int_atom=row['int_atom'], atomic_radius=row['atomic_radius'], atomic_number=row['atomic_number'], atomic_mass=row['atomic_mass'], vanderwaalsradius=row['vanderwaalsradius'], covalenzradius=row['covalenzradius'], electronegativity=row['electronegativity'], ionization_energy=row['ionization_energy'])

In [ ]:
def _worker(item, df_bonds, df_structures, molecule_info, has_y):
    
    idx = item[0]
    molecule_name = item[1]
    
    # point attribute
    x = torch.tensor(df_structures[df_structures['molecule_name']==molecule_name].sort_values(by=['atom_index'])[['int_atom','atomic_radius', 'atomic_number', 'atomic_mass', 'vanderwaalsradius', 'covalenzradius', 'electronegativity', 'ionization_energy']].values, dtype=torch.float)
    # position
    pos = torch.tensor(df_structures[df_structures['molecule_name']==molecule_name].sort_values(by=['atom_index'])[['x','y','z']].values, dtype=torch.float)

    edge_index = []
    edge_attr = []

    df_molecule_i = df_bonds[df_bonds['molecule_name']==molecule_name]
    molecule_info_i = molecule_info[molecule_info['molecule_name']==molecule_name]

    if has_y:
        y = (molecule_info_i[['potential_energy','rms']].values-y_mean)/y_std
        y = torch.tensor(y, dtype=torch.float)

    for idx, row in df_molecule_i.iterrows():
        edge_index.append([row['atom_index_0'], row['atom_index_1']])
        edge_index.append([row['atom_index_1'], row['atom_index_0']])
        edge_attr.append([row['type0'], row['int_type1']])
        edge_attr.append([row['type0'], row['int_type1']])

    edge_index = torch.tensor(edge_index, dtype=torch.long)
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)
    if has_y:
        data = Data(x=x, edge_index=edge_index.t().contiguous(), pos=pos, edge_attr=edge_attr, y=y)
    else:
        data = Data(x=x, edge_index=edge_index.t().contiguous(), pos=pos, edge_attr=edge_attr)
    d_ =  {'idx':idx, 'molecule_name':molecule_name, 'data':data}
#     print(d_)
    return d_


def generate_datalist(df_bonds, df_structures, molecule_info, molecule_names, has_y = True):
    N_ = len(molecule_names)
    segments = N_//100
    data_list = []
    
#     data_list = Parallel(_worker,{'df_bonds':df_bonds, 'df_structures':df_structures, 'molecule_info':molecule_info, 'has_y':has_y}).run(zip(np.arange(N_), molecule_names), n_jobs=4)
    
#     for start in tqdm_notebook(range(0, N_, segments)):
#         end = start + segments if start + segments < N_ else N_
#         res = Parallel(_worker,{'df_bonds':df_bonds, 'df_structures':df_structures, 'molecule_info':molecule_info, 'has_y':has_y}).run(zip(np.arange(start,end,1), molecule_names[start:end]), n_jobs=4)
    for item in tqdm_notebook(enumerate(molecule_names), total=N_):
        d_ = _worker(item, **{'df_bonds':df_bonds, 'df_structures':df_structures, 'molecule_info':molecule_info, 'has_y':has_y})
        data_list.append(d_)
    df_ = pd.DataFrame(data_list)
    df_ = df_.sort_values(by=['idx']).reset_index(drop=True)
    return df_
    atomic_radius = {'H': 0.38, 'C': 0.77, 'N': 0.75, 'O': 0.73, 'F': 0.71}
atomic_number = {'H': 1, 'C': 6, 'N': 7, 'O': 8, 'F': 9}
atomic_mass = {'H': 1.0079, 'C': 12.0107, 'N': 14.0067, 'O': 15.9994, 'F': 18.9984}
vanderwaalsradius = {'H': 120, 'C': 185, 'N': 154, 'O': 140, 'F': 135}
covalenzradius = {'H': 30, 'C': 77, 'N': 70, 'O': 66, 'F': 58}
electronegativity = {'H': 2.2, 'C': 2.55, 'N': 3.04, 'O': 3.44, 'F': 3.98}
ionization_energy = {'H': 13.5984, 'C': 11.2603, 'N': 14.5341, 'O': 13.6181, 'F': 17.4228}

train = pd.read_csv(f'{file_folder}/train.csv')
test = pd.read_csv(f'{file_folder}/test.csv')
sub = pd.read_csv(f'{file_folder}/sample_submission.csv')
train['type0'] = train['type'].apply(lambda x: int(x[0]))
train['type1'] = train['type'].apply(lambda x: x[1:])

le = LabelEncoder()
le.fit(train.type1.tolist())
int_bond_type = le.transform(train.type1.tolist()) 
train['int_type1']= int_bond_type

test['type0'] = test['type'].apply(lambda x: int(x[0]))
test['type1'] = test['type'].apply(lambda x: x[1:])
int_bond_type = le.transform(test.type1.tolist()) 
test['int_type1']= int_bond_type

le.classes_

magnetic_shielding_tensors = pd.read_csv(f'{file_folder}/magnetic_shielding_tensors.csv')
dipole_moments = pd.read_csv(f'{file_folder}/dipole_moments.csv')
mulliken_charges = pd.read_csv(f'{file_folder}/mulliken_charges.csv')
potential_energy = pd.read_csv(f'{file_folder}/potential_energy.csv')
scalar_coupling_contributions = pd.read_csv(f'{file_folder}/scalar_coupling_contributions.csv')
structures = pd.read_csv(f'{file_folder}/structures.csv')

x = magnetic_shielding_tensors.columns.values[2:]
x = magnetic_shielding_tensors[x].values
x = x.reshape(-1,3,3)
x = x + np.transpose(x,(0,2,1))
x = 0.5 * x
w, v = np.linalg.eigh(x)

sigma_iso = np.sum(w, axis=1)/3 
omega = w[:,2] - w[:,0]
kappa = 3 * (sigma_iso - w[:,1])/omega

magnetic_shielding_parameters = magnetic_shielding_tensors[magnetic_shielding_tensors.columns.values[:2]]
magnetic_shielding_parameters = pd.DataFrame(magnetic_shielding_parameters)
magnetic_shielding_parameters["sigma_iso"] = sigma_iso
magnetic_shielding_parameters["omega"] = omega
magnetic_shielding_parameters["kappa"] = kappa

atomic_radius = {'H': 0.38, 'C': 0.77, 'N': 0.75, 'O': 0.73, 'F': 0.71}
atomic_number = {'H': 1, 'C': 6, 'N': 7, 'O': 8, 'F': 9}
atomic_mass = {'H': 1.0079, 'C': 12.0107, 'N': 14.0067, 'O': 15.9994, 'F': 18.9984}
vanderwaalsradius = {'H': 120, 'C': 185, 'N': 154, 'O': 140, 'F': 135}
covalenzradius = {'H': 30, 'C': 77, 'N': 70, 'O': 66, 'F': 58}
electronegativity = {'H': 2.2, 'C': 2.55, 'N': 3.04, 'O': 3.44, 'F': 3.98}
ionization_energy = {'H': 13.5984, 'C': 11.2603, 'N': 14.5341, 'O': 13.6181, 'F': 17.4228}

structures['atomic_radius'] = structures['atom'].apply(lambda x: atomic_radius[x])
structures['atomic_number'] = structures['atom'].apply(lambda x: atomic_number[x])
structures['atomic_mass'] = structures['atom'].apply(lambda x: atomic_mass[x])
structures['vanderwaalsradius'] = structures['atom'].apply(lambda x: vanderwaalsradius[x])
structures['covalenzradius'] = structures['atom'].apply(lambda x: covalenzradius[x])
structures['electronegativity'] = structures['atom'].apply(lambda x: electronegativity[x])
structures['ionization_energy'] = structures['atom'].apply(lambda x: ionization_energy[x])


structures.head()

le = LabelEncoder()
le.fit(structures.atom.tolist())
int_atom = le.transform(structures.atom.tolist()) 
structures['int_atom'] = int_atom

le.classes_

molecule_names = train.molecule_name.unique().tolist()

magnetic_shielding_parameters.head()

mulliken_charges.head()

dipole_moments['rms'] = np.sqrt(dipole_moments['X']**2 + dipole_moments['Y']**2 + dipole_moments['Z']**2 )

dipole_moments.head()#MAE 3.x without scaled y, 2.x with scaled y

potential_energy.head()#MAE 30.x with scaled y

molecule_info = pd.merge(dipole_moments, potential_energy, how = 'left', left_on  = ['molecule_name'], right_on = ['molecule_name'])

molecule_info.head()

for col in molecule_info.columns.tolist():
    print(np.where(molecule_info[col].isna())[0].shape)

atom_info = pd.merge(magnetic_shielding_parameters, mulliken_charges, how = 'left', left_on  = ['molecule_name','atom_index'], right_on = ['molecule_name','atom_index'])

atom_info.head()

for col in atom_info.columns.tolist():
    print(np.where(atom_info[col].isna())[0].shape)

y_std = np.std(molecule_info[['potential_energy','rms']].values, axis=0)
y_mean = np.mean(molecule_info[['potential_energy','rms']].values, axis=0)
y_std, y_mean